<a href="https://colab.research.google.com/github/SunSlick2/MoveEmails/blob/main/Email_Migrator2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import win32com.client
import pythoncom
import logging
import time
from datetime import datetime
import json
import os
import tkinter as tk
from tkinter import messagebox

class EmailMigrator:
    def __init__(self):
        self.setup_logging()
        self.migration_report = {
            'start_time': None,
            'end_time': None,
            'destination_type': 'N/A', # New field to track the chosen destination
            'destination_path': 'N/A', # New field to track the chosen destination path
            'total_attempted': 0, # Grand total mail items found and attempted to move from ALL PSTs
            'total_successful': 0, # Grand total items whose .Move() operation succeeded across ALL PSTs
            'total_failed': 0,     # Grand total items whose .Move() operation failed across ALL PSTs
            'aggregate_validation_passed': False, # New field for overall aggregate validation
            'pst_migrations_details': [], # List to store detailed reports for each PST processed
            'failed_items_overall': [], # Consolidated list of all failed items from all PSTs
            'folder_summary_overall': {} # Consolidated summary of all folders processed
        }

    def setup_logging(self):
        """Setup comprehensive logging"""
        log_dir = "migration_logs"
        os.makedirs(log_dir, exist_ok=True)
        log_filename = os.path.join(log_dir, f'pst_migration_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

        logging.basicConfig(
            filename=log_filename,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(levelname)s: %(message)s')
        console.setFormatter(formatter)
        logging.getLogger().addHandler(console)

        file_handler = logging.FileHandler(log_filename)
        file_handler.setLevel(logging.DEBUG) # More detailed logging to file
        file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logging.getLogger().addHandler(file_handler)

        logging.info(f"Logging to: {log_filename}")

    def get_item_signature(self, item):
        """Create a unique signature for reporting/logging"""
        try:
            return {
                'subject': getattr(item, 'Subject', 'N/A'),
                'sent_on': str(getattr(item, 'SentOn', None)) if getattr(item, 'SentOn', None) else None,
                'sender': getattr(item, 'SenderName', 'N/A'),
                'entry_id': getattr(item, 'EntryID', 'N/A'),
                'size': getattr(item, 'Size', 'N/A')
            }
        except Exception as e:
            return {'subject': 'Unknown', 'error': f'Could not get item properties: {e}'}

    def get_folder_item_count(self, folder_obj):
        """Safely get the count of items in an Outlook folder."""
        try:
            return folder_obj.Items.Count
        except Exception as e:
            logging.error(f"Error getting item count for folder '{getattr(folder_obj, 'FolderPath', 'N/A')}': {e}", exc_info=True)
            return -1 # Indicate failure

    def process_folder(self, source_folder, target_folder, current_pst_report, folder_path=""):
        """
        Process all items in a folder and move them to the specified target_folder.
        Updates the current_pst_report with counts for the current PST.
        """

        current_folder_path = f"{folder_path}/{source_folder.Name}" if folder_path else source_folder.Name
        logging.info(f"Processing folder: {current_folder_path}")

        try:
            items_list = list(source_folder.Items)
            item_count = len(items_list)
            logging.info(f"Found {item_count} items in '{current_folder_path}' to process.")

            for item in reversed(items_list):
                try:
                    if item.Class == 43: # olMail item
                        current_pst_report['total_attempted_current_pst'] += 1

                        original_signature = self.get_item_signature(item)
                        logging.info(f"Attempting to move item: {original_signature['subject'][:50]}...")

                        try:
                            item.Move(target_folder)

                            current_pst_report['total_successful_current_pst'] += 1
                            logging.info(f"Successfully moved: {original_signature['subject'][:50]}...")

                        except Exception as move_error:
                            current_pst_report['total_failed_current_pst'] += 1
                            current_pst_report['failed_items_current_pst'].append({
                                'folder': current_folder_path,
                                'subject': original_signature['subject'],
                                'error': str(move_error),
                                'original_signature': original_signature
                            })
                            logging.error(f"Move failed for '{original_signature['subject']}': {move_error}", exc_info=True)

                except Exception as item_error:
                    current_pst_report['total_failed_current_pst'] += 1 # Count as a failed attempt if item access fails
                    logging.error(f"Error processing item in '{current_folder_path}': {item_error}", exc_info=True)
                    continue

        except Exception as folder_items_error:
            logging.error(f"Error accessing items in folder '{current_folder_path}': {folder_items_error}", exc_info=True)
            # If we can't access items, we count all found items (if any) as failed for this folder's iteration
            current_pst_report['total_failed_current_pst'] += item_count if 'item_count' in locals() else 0

        # Update folder summary for the current PST
        current_pst_report['folder_summary_current_pst'][current_folder_path] = {
            'successful_moves_from_this_folder': current_pst_report['total_successful_current_pst'] - sum(f['successful_moves_from_this_folder'] for f in current_pst_report['folder_summary_current_pst'].values()),
            'failed_moves_from_this_folder': current_pst_report['total_failed_current_pst'] - sum(f['failed_moves_from_this_folder'] for f in current_pst_report['folder_summary_current_pst'].values()),
            'total_items_in_source_folder': item_count if 'item_count' in locals() else 0
        }

        # Process subfolders, *but still move their items to the same target_folder*
        for subfolder in source_folder.Folders:
            try:
                # Pass the *original* target_folder and current_pst_report to subfolders
                self.process_folder(subfolder, target_folder, current_pst_report, current_folder_path)
            except Exception as subfolder_error:
                logging.error(f"Error processing subfolder '{subfolder.Name}': {subfolder_error}", exc_info=True)
                current_pst_report['total_failed_current_pst'] += 1

        # Return nothing, as current_pst_report is modified in-place

    def generate_report(self):
        """Generate a comprehensive migration report"""
        report_dir = "migration_reports"
        os.makedirs(report_dir, exist_ok=True)
        report_filename = os.path.join(report_dir, f'migration_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json')

        self.migration_report['end_time'] = datetime.now().isoformat()
        if self.migration_report['start_time']:
            self.migration_report['duration_seconds'] = (
                datetime.fromisoformat(self.migration_report['end_time']) -
                datetime.fromisoformat(self.migration_report['start_time'])
            ).total_seconds()
        else:
            self.migration_report['duration_seconds'] = 0

        self.migration_report['success_rate'] = (
            self.migration_report['total_successful'] / self.migration_report['total_attempted'] * 100
        ) if self.migration_report['total_attempted'] > 0 else 0

        with open(report_filename, 'w') as f:
            json.dump(self.migration_report, f, indent=2, default=str)

        logging.info(f"Migration report saved to: {report_filename}")

        # Print summary to console
        print(f"\n=== OVERALL MIGRATION SUMMARY ===")
        print(f"Destination Type: {self.migration_report['destination_type']}")
        print(f"Destination Path: {self.migration_report['destination_path']}")
        print(f"Start Time: {self.migration_report['start_time']}")
        print(f"End Time: {self.migration_report['end_time']}")
        print(f"Duration: {self.migration_report['duration_seconds']:.2f} seconds")
        print(f"Grand Total PST Items Found (Attempted to Move): {self.migration_report['total_attempted']}")
        print(f"Grand Total Successful .Move() Operations: {self.migration_report['total_successful']}")
        print(f"Grand Total Failed .Move() Operations: {self.migration_report['total_failed']}")
        print(f"Overall Success Rate (of .Move() calls): {self.migration_report['success_rate']:.2f}%")
        print(f"Aggregate Validation Passed: {self.migration_report['aggregate_validation_passed']}")

        if self.migration_report['failed_items_overall']:
            print(f"\n=== OVERALL FAILED ITEMS ({len(self.migration_report['failed_items_overall'])}) ===")
            print("Showing first 10 overall failures (check log file for all details):")
            for i, failed_item in enumerate(self.migration_report['failed_items_overall'][:10], 1):
                print(f"{i}. PST: {failed_item.get('pst_display_name', 'N/A')}, Subject: {failed_item['subject'][:70]}...")
                print(f"   From Folder: {failed_item['folder']}")
                print(f"   Error: {failed_item['error']}")
                print("-" * 30)

        print(f"\n=== INDIVIDUAL PST MIGRATION DETAILS ===")
        if not self.migration_report['pst_migrations_details']:
            print("No PSTs were processed.")
        else:
            for pst_detail in self.migration_report['pst_migrations_details']:
                print(f"\n--- PST: '{pst_detail['pst_display_name']}' (Path: {pst_detail['pst_file_path']}) ---")
                print(f"  Items Attempted: {pst_detail['total_attempted_current_pst']}")
                print(f"  Successful Moves: {pst_detail['total_successful_current_pst']}")
                print(f"  Failed Moves: {pst_detail['total_failed_current_pst']}")
                pst_success_rate = (
                    pst_detail['total_successful_current_pst'] / pst_detail['total_attempted_current_pst'] * 100
                ) if pst_detail['total_attempted_current_pst'] > 0 else 0
                print(f"  Success Rate: {pst_success_rate:.2f}%")

                if pst_detail['failed_items_current_pst']:
                    print(f"  Failed items for this PST ({len(pst_detail['failed_items_current_pst'])}):")
                    for i, failed_item in enumerate(pst_detail['failed_items_current_pst'][:5], 1):
                        print(f"    {i}. Subject: {failed_item['subject'][:70]}... - {failed_item['error']}")
                else:
                    print("  No failed items for this PST.")
                print("-" * 50)


    def select_pst_store(self, namespace):
        """Identifies and returns a list of all open PST stores."""
        pst_stores = []
        for store in namespace.Stores:
            if hasattr(store, 'FilePath') and store.FilePath and store.FilePath.lower().endswith('.pst'):
                pst_stores.append(store)

        if not pst_stores:
            logging.error("No PST files found currently open in Outlook.")
            messagebox.showerror("No PSTs Open", "No PST files are currently open in Outlook. Please open the desired PST file(s) in Outlook first, then run this script.")
            return None # Return None if no PSTs are open

        print("\n--- Detected PST Files Open in Outlook ---")
        for i, store in enumerate(pst_stores):
            print(f"{i + 1}. {store.DisplayName} (Path: {getattr(store, 'FilePath', 'N/A')})")

        return pst_stores # Return the list of all detected PSTs

    def select_target_inbox(self, namespace):
        """
        Prompts the user to select the target Inbox for an OST-linked account.
        Returns the target_inbox Folder object and the selected Account object.
        """
        accounts = []
        for account in namespace.Accounts:
            if account.AccountType == 0 or account.AccountType == 5:
                accounts.append(account)

        if not accounts:
            logging.error("No Exchange, Office 365, or IMAP accounts found in Outlook.")
            messagebox.showerror("No Accounts Found", "No Exchange, Office 365, or IMAP accounts found in Outlook. Please ensure an appropriate account is configured.")
            return None, None, None

        print("\n--- Select Target Inbox (OST-linked Account) ---")
        display_list = []
        for i, account in enumerate(accounts):
            try:
                store = account.DeliveryStore
                root_folder = store.GetRootFolder()
                inbox = root_folder.Folders.Item("Inbox")
                display_list.append((account, inbox))
                print(f"{len(display_list)}. {account.DisplayName} (Inbox: {inbox.FolderPath})")
            except Exception as e:
                logging.warning(f"Error accessing Inbox for account '{account.DisplayName}': {e}")

        if not display_list:
            logging.error("Could not find any selectable Inboxes for OST-linked accounts.")
            messagebox.showerror("No Inboxes Found", "Could not find any selectable Inboxes for OST-linked accounts. Check your Outlook configuration.")
            return None, None, None

        while True:
            try:
                choice = input("Enter the number of the account whose Inbox you want to use as the target: ")
                selected_index = int(choice) - 1

                if 0 <= selected_index < len(display_list):
                    selected_account, target_inbox = display_list[selected_index]
                    logging.info(f"Target Inbox selected: {target_inbox.FolderPath}")
                    return target_inbox, selected_account.DisplayName, target_inbox.FolderPath
                else:
                    print("Invalid choice. Please enter a valid number from the list.")
            except ValueError:
                print("Invalid input. Please enter a number.")
            except Exception as e:
                print(f"Error during target inbox selection: {e}. Please try again.")
                logging.error(f"Unexpected error during target inbox selection: {e}", exc_info=True)
                return None, None, None

    def select_target_archive(self, namespace):
        """
        Prompts the user to select the target Online Archive account.
        Returns the root folder of the selected archive.
        """
        archive_stores = []
        for store in namespace.Stores:
            try:
                # 11 is the constant for olStoreExchangePublicFolder
                # Added more robust checks for archive types
                if hasattr(store, 'IsArchive') and store.IsArchive:
                    logging.info(f"Found store '{store.DisplayName}' with IsArchive property set to True.")
                    archive_stores.append(store)
                elif hasattr(store, 'ExchangeStoreType') and store.ExchangeStoreType == 11:
                    logging.info(f"Found store '{store.DisplayName}' with ExchangeStoreType = 11 (olStoreExchangePublicFolder), treating as an archive.")
                    archive_stores.append(store)
                else:
                    logging.debug(f"Store '{store.DisplayName}' is not an archive.")
            except Exception as e:
                logging.debug(f"Could not check properties for store '{store.DisplayName}': {e}")

        if not archive_stores:
            logging.error("No Online Archive accounts found in Outlook.")
            messagebox.showerror("No Online Archive Found", "No Online Archive accounts found in Outlook. Please ensure an archive is configured and open.")
            return None, None, None

        print("\n--- Select Target Online Archive ---")
        display_list = []
        for i, store in enumerate(archive_stores):
            try:
                archive_root_folder = store.GetRootFolder()
                display_list.append((store, archive_root_folder))
                print(f"{len(display_list)}. {store.DisplayName} (Path: {archive_root_folder.FolderPath})")
            except Exception as e:
                logging.warning(f"Error accessing root folder for archive '{store.DisplayName}': {e}")

        if not display_list:
            logging.error("Could not find any selectable Online Archives.")
            messagebox.showerror("No Online Archives Found", "Could not find any selectable Online Archives. Check your Outlook configuration.")
            return None, None, None

        while True:
            try:
                choice = input("Enter the number of the Online Archive to use as the target: ")
                selected_index = int(choice) - 1

                if 0 <= selected_index < len(display_list):
                    selected_store, target_archive_root = display_list[selected_index]
                    logging.info(f"Target Online Archive selected: {target_archive_root.FolderPath}")
                    return target_archive_root, selected_store.DisplayName, target_archive_root.FolderPath
                else:
                    print("Invalid choice. Please enter a valid number from the list.")
            except ValueError:
                print("Invalid input. Please enter a number.")
            except Exception as e:
                print(f"Error during target archive selection: {e}. Please try again.")
                logging.error(f"Unexpected error during target archive selection: {e}", exc_info=True)
                return None, None, None

    def select_target_pst(self, namespace, source_pst_file_paths):
        """
        Prompts the user to select a target PST from all open PSTs,
        excluding the ones being used as a source.
        """
        all_pst_stores = self.select_pst_store(namespace)
        if not all_pst_stores:
            return None, None, None

        target_psts = [
            store for store in all_pst_stores
            if getattr(store, 'FilePath', 'N/A') not in source_pst_file_paths
        ]

        if not target_psts:
            logging.error("No other PST files available as a target.")
            messagebox.showerror("No Target PST Found", "No other PST files are open in Outlook that can be used as a target. Please open another PST file.")
            return None, None, None

        print("\n--- Select Target PST ---")
        for i, store in enumerate(target_psts):
            print(f"{i + 1}. {store.DisplayName} (Path: {getattr(store, 'FilePath', 'N/A')})")

        while True:
            try:
                choice = input("Enter the number of the PST to use as the target: ")
                selected_index = int(choice) - 1

                if 0 <= selected_index < len(target_psts):
                    target_store = target_psts[selected_index]
                    target_root_folder = target_store.GetRootFolder()
                    logging.info(f"Target PST selected: {target_root_folder.FolderPath}")
                    return target_root_folder, target_store.DisplayName, target_root_folder.FolderPath
                else:
                    print("Invalid choice. Please enter a valid number from the list.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def select_destination_type(self):
        """Prompts the user to select the migration destination."""
        print("\n--- Select Migration Destination ---")
        print("1. OST/Exchange Inbox")
        print("2. Online Archive")
        print("3. Another PST File")

        while True:
            try:
                choice = input("Enter the number of your choice (1-3): ")
                if choice in ['1', '2', '3']:
                    return int(choice)
                else:
                    print("Invalid choice. Please enter 1, 2, or 3.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def run_migration(self):
        """Main migration function with comprehensive validation for multiple PSTs"""
        logging.info("Starting PST to Destination migration.")
        self.migration_report['start_time'] = datetime.now().isoformat()

        pythoncom.CoInitialize()
        outlook = None
        all_pst_stores = []
        target_folder = None
        target_display_name = None
        target_path = None

        try:
            outlook = win32com.client.Dispatch("Outlook.Application")
            namespace = outlook.GetNamespace("MAPI")

            # --- Step 1: Detect all open PSTs ---
            all_pst_stores = self.select_pst_store(namespace)
            if not all_pst_stores:
                logging.error("No PST stores found to process. Exiting migration.")
                return False

            logging.info(f"Detected {len(all_pst_stores)} PSTs for migration.")
            source_pst_file_paths = [getattr(store, 'FilePath', 'N/A') for store in all_pst_stores]

            # --- Step 2: User selects the destination type ---
            destination_choice = self.select_destination_type()

            # --- Step 3: User selects the specific target based on type ---
            if destination_choice == 1:
                target_folder, target_display_name, target_path = self.select_target_inbox(namespace)
                self.migration_report['destination_type'] = "OST Inbox"
            elif destination_choice == 2:
                target_folder, target_display_name, target_path = self.select_target_archive(namespace)
                self.migration_report['destination_type'] = "Online Archive"
            elif destination_choice == 3:
                target_folder, target_display_name, target_path = self.select_target_pst(namespace, source_pst_file_paths)
                self.migration_report['destination_type'] = "PST File"

            if not target_folder:
                logging.error("Target destination selection failed. Exiting migration.")
                return False

            self.migration_report['destination_path'] = target_path

            # --- Step 4: Get initial count of items in the target folder ---
            initial_target_item_count = self.get_folder_item_count(target_folder)
            if initial_target_item_count == -1:
                logging.error("Failed to get initial item count for target destination. Cannot perform aggregate validation.")
                messagebox.showerror("Validation Error", "Failed to get initial item count. Check logs.")
                return False
            logging.info(f"Initial item count in target destination ('{target_path}'): {initial_target_item_count}")
            print(f"\nInitial item count in target destination: {initial_target_item_count}")

            print("\n" + "=" * 50)
            print(f"CONFIRMATION: You are about to MOVE ALL emails from {len(all_pst_stores)} selected PST(s)")
            print(f"into the '{self.migration_report['destination_type']}' of '{target_display_name}' ('{target_path}').")
            print("This action will flatten the folder structures of ALL PSTs into the single target folder.")
            print("THIS ACTION CANNOT BE UNDONE. ENSURE YOU HAVE BACKUPS OF ALL YOUR PST FILES.")
            print("=" * 50)

            confirm = input("Type 'CONFIRM' to proceed with the migration: ")
            if confirm.strip().upper() != 'CONFIRM':
                logging.warning("Migration cancelled by user.")
                print("Operation cancelled.")
                return False

            # --- Step 5: Process each PST ---
            for i, pst_store_obj in enumerate(all_pst_stores):
                pst_store_display_name = pst_store_obj.DisplayName
                pst_file_path = getattr(pst_store_obj, 'FilePath', 'N/A')

                logging.info(f"\n--- Processing PST {i+1}/{len(all_pst_stores)}: '{pst_store_display_name}' (Path: {pst_file_path}) ---")
                print(f"\n--- Processing PST {i+1}/{len(all_pst_stores)}: '{pst_store_display_name}' ---")

                # Initialize a temporary report for the current PST
                current_pst_report = {
                    'pst_display_name': pst_store_display_name,
                    'pst_file_path': pst_file_path,
                    'total_attempted_current_pst': 0,
                    'total_successful_current_pst': 0,
                    'total_failed_current_pst': 0,
                    'failed_items_current_pst': [],
                    'folder_summary_current_pst': {}
                }

                # Process all folders in the current PST, moving items to the target_folder
                self.process_folder(
                    pst_store_obj.GetRootFolder(),
                    target_folder,
                    current_pst_report
                )

                # Consolidate individual PST report into overall report
                self.migration_report['pst_migrations_details'].append(current_pst_report)
                self.migration_report['total_attempted'] += current_pst_report['total_attempted_current_pst']
                self.migration_report['total_successful'] += current_pst_report['total_successful_current_pst']
                self.migration_report['total_failed'] += current_pst_report['total_failed_current_pst']
                self.migration_report['failed_items_overall'].extend(
                    [{**item, 'pst_display_name': pst_store_display_name} for item in current_pst_report['failed_items_current_pst']]
                )
                self.migration_report['folder_summary_overall'].update(current_pst_report['folder_summary_current_pst'])

            logging.info(f"\nFinished moving items from all {len(all_pst_stores)} PSTs.")
            print(f"\nFinished moving items from all {len(all_pst_stores)} PSTs.")

            # --- Step 6: Post-migration wait for synchronization ---
            logging.info(f"Waiting for 60 seconds to allow target destination to synchronize...")
            print("Waiting for 60 seconds for target to synchronize...")
            time.sleep(60)
            logging.info("Wait complete.")

            # --- Step 7: Post-migration count and aggregate validation ---
            final_target_item_count = self.get_folder_item_count(target_folder)
            if final_target_item_count == -1:
                logging.error("Failed to get final item count for target destination. Aggregate validation cannot be completed.")
                messagebox.showerror("Validation Error", "Failed to get final item count. Check logs.")
                self.migration_report['aggregate_validation_passed'] = False
            else:
                logging.info(f"Final item count in target destination ('{target_path}'): {final_target_item_count}")
                print(f"Final item count in target destination: {final_target_item_count}")

                delta_items = final_target_item_count - initial_target_item_count
                logging.info(f"Delta items in target: {delta_items}. Total successful moves across all PSTs: {self.migration_report['total_successful']}")

                if (final_target_item_count > initial_target_item_count and
                    delta_items >= self.migration_report['total_successful']):
                    self.migration_report['aggregate_validation_passed'] = True
                    logging.info("Aggregate count validation PASSED: Target increased by expected amount.")
                else:
                    self.migration_report['aggregate_validation_passed'] = False
                    logging.warning(
                        f"Aggregate count validation FAILED. "
                        f"Expected increase >= {self.migration_report['total_successful']}, but got {delta_items}. "
                        f"(Initial: {initial_target_item_count}, Final: {final_target_item_count})"
                    )
                    messagebox.showwarning(
                        "Aggregate Validation Failed",
                        f"Count validation failed.\n"
                        f"Expected total increase from all PSTs: >= {self.migration_report['total_successful']}\n"
                        f"Actual total increase in destination: {delta_items}\n"
                        f"Please check Outlook and logs for discrepancies."
                    )

            # Generate final comprehensive report
            self.generate_report()

            overall_success = (self.migration_report['total_failed'] == 0 and self.migration_report['aggregate_validation_passed'])

            if overall_success:
                logging.info("Migration completed successfully with 0 errors and passed aggregate validation!")
            else:
                logging.warning(f"Migration completed with {self.migration_report['total_failed']} errors. Aggregate validation: {self.migration_report['aggregate_validation_passed']}. Please check the log and report files.")

            return overall_success

        except Exception as e:
            logging.error(f"Critical error during migration: {e}", exc_info=True)
            messagebox.showerror("Migration Error", f"A critical error occurred during migration. Check logs for details: {e}")
            return False
        finally:
            if outlook:
                del outlook
            pythoncom.CoUninitialize()

# Usage
if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("   PST to Destination Email Migration Tool (All Open PSTs)   ")
    print("=" * 60)
    print("This tool requires you to **manually open the desired PST file(s) in Outlook first**.")
    print("It will then list all these open PSTs and process them sequentially,")
    print("moving emails from all their folders to a single selected destination.")
    print("\n" + "=" * 60)

    migrator = EmailMigrator()
    success = migrator.run_migration()

    if success:
        print("\n✅ Migration completed successfully for all PSTs and passed aggregate validation!")
    else:
        print("\n❌ Migration completed with errors or failed aggregate validation. Check 'migration_logs' and 'migration_reports' folders for details.")